# 🔬 ResNet50 for Diabetic Retinopathy Classification

## 📋 Experiment Overview
- **Model**: ResNet50 with Fine-Tuning (Low Learning Rate)
- **Task**: Binary classification (No DR vs Has DR)
- **Dataset**: Kaggle Diabetic Retinopathy (35,126 images)
- **Strategy**: Transfer learning with gradual unfreezing
- **Expected AUC**: 0.75+ (better than VGG16's 0.706)

## 🎯 Key Improvements over VGG16:
1. **Modern architecture** with skip connections
2. **Focal loss** to handle class imbalance
3. **Optimized thresholds** for medical recall
4. **Two-stage training** (frozen → fine-tuned)

In [ ]:
# ============================================
# STEP 1: IMPORT LIBRARIES
# ============================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pathlib import Path

# Deep Learning
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, roc_curve, precision_recall_curve
)
from sklearn.utils.class_weight import compute_class_weight

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("📦 All libraries imported successfully!")
print(f"🔥 TensorFlow version: {tf.__version__}")
print(f"🎯 GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# ============================================
# STEP 2: KAGGLE SETUP & DATA LOADING
# ============================================
# For Kaggle - adjust paths as needed
KAGGLE_INPUT_PATH = '/kaggle/input/diabetic-retinopathy-resized'
LOCAL_PATH = '/Users/muhirwa/Desktop/projects/diabetes_retinopathy/data'

# Try Kaggle path first, fallback to local
if os.path.exists(KAGGLE_INPUT_PATH):
    DATA_PATH = KAGGLE_INPUT_PATH
    print("🌐 Using Kaggle dataset path")
elif os.path.exists(LOCAL_PATH):
    DATA_PATH = LOCAL_PATH
    print("💻 Using local dataset path")
else:
    print("❌ Dataset not found! Please check paths.")
    DATA_PATH = None

print(f"📁 Data path: {DATA_PATH}")

# Load labels
if DATA_PATH:
    labels_path = os.path.join(DATA_PATH, 'trainLabels.csv')
    if os.path.exists(labels_path):
        labels_df = pd.read_csv(labels_path)
        print(f"📊 Labels loaded! Total images: {len(labels_df)}")
        print("\nFirst 5 rows:")
        print(labels_df.head())
    else:
        print(f"❌ Labels file not found at {labels_path}")

In [ ]:
# ============================================
# STEP 3: BINARY CLASSIFICATION SETUP
# ============================================
print("\n" + "="*50)
print("STEP 3: Convert to Binary Classification")
print("="*50)

# Convert to binary: 0 = No DR, 1+ = Has DR
labels_df['binary_label'] = (labels_df['level'] > 0).astype(int)

print("\n🎯 Binary classification:")
print(labels_df['binary_label'].value_counts())
print(f"  No DR (0): {(labels_df['binary_label']==0).sum()} images ({(labels_df['binary_label']==0).sum()/len(labels_df)*100:.1f}%)")
print(f"  Has DR (1): {(labels_df['binary_label']==1).sum()} images ({(labels_df['binary_label']==1).sum()/len(labels_df)*100:.1f}%)")

# Add file paths
image_dir = os.path.join(DATA_PATH, 'resized_train_cropped', 'resized_train_cropped')
labels_df['filepath'] = labels_df['image'].apply(lambda x: os.path.join(image_dir, f"{x}.jpeg"))

# Check if files exist (sample check)
sample_files = labels_df['filepath'].head(5).tolist()
existing_files = [os.path.exists(f) for f in sample_files]
print(f"\n📁 Sample files exist: {sum(existing_files)}/{len(existing_files)}")
if sum(existing_files) == 0:
    print("⚠️  Check image directory path!")
    print(f"Looking in: {image_dir}")

In [ ]:
# ============================================
# STEP 4: TRAIN/VAL/TEST SPLITS
# ============================================
print("\n" + "="*60)
print("STEP 4: CREATE TRAIN/VAL/TEST SPLITS")
print("="*60)

# Use all images for ResNet50 (more data = better performance)
labels_df_sample = labels_df.copy()
print(f"Using all {len(labels_df_sample)} images")

# Split: 70% train, 15% validation, 15% test
train_df, temp_df = train_test_split(
    labels_df_sample,
    test_size=0.3,
    random_state=42,
    stratify=labels_df_sample['binary_label']  # Maintain class distribution
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['binary_label']
)

print("\n📊 Data split complete:")
print(f"  Train:      {len(train_df):6d} images ({len(train_df)/len(labels_df_sample)*100:.1f}%)")
print(f"  Validation: {len(val_df):6d} images ({len(val_df)/len(labels_df_sample)*100:.1f}%)")
print(f"  Test:       {len(test_df):6d} images ({len(test_df)/len(labels_df_sample)*100:.1f}%)")

# Check class distribution in each split
print("\n📊 Class distribution per split:")
for name, df in [('Train', train_df), ('Val', val_df), ('Test', test_df)]:
    class_counts = df['binary_label'].value_counts()
    print(f"  {name:5s}: No DR = {class_counts[0]:4d} ({class_counts[0]/len(df)*100:.1f}%), Has DR = {class_counts[1]:4d} ({class_counts[1]/len(df)*100:.1f}%)")

In [ ]:
# ============================================
# STEP 5: DATA AUGMENTATION & GENERATORS
# ============================================
print("\n" + "="*60)
print("STEP 5: DATA AUGMENTATION & GENERATORS")
print("="*60)

# Enhanced augmentation for medical images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,          # Slightly more rotation for retinal images
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,       # Retinal images can be flipped
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Validation and test (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Convert labels to strings (required by flow_from_dataframe)
train_df['binary_label_str'] = train_df['binary_label'].astype(str)
val_df['binary_label_str'] = val_df['binary_label'].astype(str)
test_df['binary_label_str'] = test_df['binary_label'].astype(str)

# Create generators
BATCH_SIZE = 32  # Optimal for most GPUs
IMG_SIZE = (224, 224)  # ResNet50 input size

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filepath',
    y_col='binary_label_str',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=42
)

val_generator = val_test_datagen.flow_from_dataframe(
    val_df,
    x_col='filepath',
    y_col='binary_label_str',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_dataframe(
    test_df,
    x_col='filepath',
    y_col='binary_label_str',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print(f"\n✅ Generators created successfully!")
print(f"   Train batches: {len(train_generator)}")
print(f"   Val batches:   {len(val_generator)}")
print(f"   Test batches:  {len(test_generator)}")

In [ ]:
# ============================================
# STEP 6: FOCAL LOSS FOR CLASS IMBALANCE
# ============================================
print("\n" + "="*60)
print("STEP 6: FOCAL LOSS IMPLEMENTATION")
print("="*60)

def focal_loss(gamma=2.0, alpha=0.75):
    """
    Focal Loss for handling class imbalance.
    
    Args:
        gamma: Focusing parameter (higher = more focus on hard examples)
        alpha: Weighting factor for minority class
    
    Returns:
        Focal loss function for Keras
    """
    def focal_loss_fixed(y_true, y_pred):
        # Ensure types
        y_true = tf.cast(y_true, tf.float32)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        # Calculate p_t
        p_t = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        
        # Calculate alpha_t
        alpha_factor = tf.ones_like(y_true) * alpha
        alpha_t = tf.where(tf.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        
        # Calculate focal weight
        cross_entropy = -tf.math.log(p_t)
        weight = alpha_t * tf.pow((1 - p_t), gamma)
        
        # Final loss
        loss = weight * cross_entropy
        return tf.reduce_mean(loss)
    
    return focal_loss_fixed

print("✅ Focal loss function defined!")
print(f"   Gamma (focusing): {2.0}")
print(f"   Alpha (weighting): {0.75}")
print("   📋 This will help the model focus on hard-to-classify minority cases")

In [ ]:
# ============================================
# STEP 7: BUILD RESNET50 MODEL
# ============================================
print("\n" + "="*60)
print("STEP 7: BUILDING RESNET50 MODEL")
print("="*60)

# Load pretrained ResNet50
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Start with frozen base model (Stage 1)
base_model.trainable = False

# Build model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    
    # Enhanced classifier head
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu'),
    Dropout(0.3),
    
    Dense(1, activation='sigmoid')  # Binary classification
])

print(f"🔒 ResNet50 base layers frozen (Transfer learning stage)")
print(f"\n📊 Model Summary:")
model.summary()

# Count parameters
total_params = model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"\n✅ Total parameters: {total_params:,}")
print(f"✅ Trainable parameters: {trainable_params:,}")
print(f"✅ Frozen parameters: {total_params - trainable_params:,}")

In [ ]:
# ============================================
# STEP 8: CALLBACKS & CLASS WEIGHTS
# ============================================
print("\n" + "="*60)
print("STEP 8: CALLBACKS & CLASS WEIGHTS")
print("="*60)

# Calculate class weights
y_train_labels = train_df['binary_label'].astype(int).values
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"⚖️  Class weights calculated:")
print(f"   No DR (0): {class_weight_dict[0]:.3f}")
print(f"   Has DR (1): {class_weight_dict[1]:.3f}")
print(f"   (Higher weight = more important during training)")

# Define callbacks
callbacks_stage1 = [
    EarlyStopping(
        monitor='val_auc',
        patience=5,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    
    ReduceLROnPlateau(
        monitor='val_auc',
        factor=0.5,
        patience=3,
        mode='max',
        min_lr=1e-7,
        verbose=1
    ),
    
    ModelCheckpoint(
        'best_resnet50_stage1.h5',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

print(f"\n✅ Callbacks configured for Stage 1 (frozen base)")
print(f"   📊 Monitoring: val_auc (maximize)")
print(f"   ⏹️  Early stopping: 5 epochs patience")
print(f"   📉 Learning rate reduction: 3 epochs patience")

In [ ]:
# ============================================
# STEP 9: STAGE 1 TRAINING (FROZEN BASE)
# ============================================
print("\n" + "="*60)
print("STEP 9: STAGE 1 TRAINING (FROZEN RESNET50)")
print("="*60)

# Compile model for Stage 1
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Higher LR for frozen base
    loss=focal_loss(gamma=2.0, alpha=0.75),
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

EPOCHS_STAGE1 = 10
print(f"⏳ Stage 1: Training classifier head for {EPOCHS_STAGE1} epochs...")
print(f"📊 Learning rate: 0.001 (higher for new layers)")
print(f"🔒 ResNet50 base: FROZEN")

start_time = time.time()

history_stage1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_STAGE1,
    callbacks=callbacks_stage1,
    verbose=1,
    class_weight=class_weight_dict
)

stage1_time = time.time() - start_time
print(f"\n✅ Stage 1 completed in {stage1_time:.2f} seconds ({stage1_time/60:.2f} minutes)")

# Get Stage 1 results
stage1_val_auc = max(history_stage1.history['val_auc'])
print(f"📊 Best Stage 1 validation AUC: {stage1_val_auc:.4f}")

In [ ]:
# ============================================
# STEP 10: STAGE 2 SETUP (FINE-TUNING)
# ============================================
print("\n" + "="*60)
print("STEP 10: STAGE 2 SETUP (FINE-TUNING)")
print("="*60)

# Unfreeze ResNet50 layers gradually
base_model.trainable = True

# Fine-tune from this layer onwards (last 30 layers)
fine_tune_at = len(base_model.layers) - 30

# Freeze early layers, unfreeze later layers
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"🔓 ResNet50 fine-tuning enabled")
print(f"   Total layers: {len(base_model.layers)}")
print(f"   Frozen layers: {fine_tune_at} (early layers)")
print(f"   Trainable layers: {len(base_model.layers) - fine_tune_at} (later layers)")

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # MUCH lower LR for fine-tuning
    loss=focal_loss(gamma=2.0, alpha=0.75),
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Update callbacks for Stage 2
callbacks_stage2 = [
    EarlyStopping(
        monitor='val_auc',
        patience=7,  # More patience for fine-tuning
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    
    ReduceLROnPlateau(
        monitor='val_auc',
        factor=0.5,
        patience=3,
        mode='max',
        min_lr=1e-7,
        verbose=1
    ),
    
    ModelCheckpoint(
        'best_resnet50_stage2.h5',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )
]

# Count trainable parameters after unfreezing
trainable_params_stage2 = sum([tf.size(w).numpy() for w in model.trainable_weights])
print(f"\n✅ Stage 2 parameters:")
print(f"   Trainable: {trainable_params_stage2:,}")
print(f"   Learning rate: 0.0001 (10x lower than Stage 1)")

In [ ]:
# ============================================
# STEP 11: STAGE 2 TRAINING (FINE-TUNING)
# ============================================
print("\n" + "="*60)
print("STEP 11: STAGE 2 TRAINING (FINE-TUNING)")
print("="*60)

EPOCHS_STAGE2 = 15
print(f"⏳ Stage 2: Fine-tuning for {EPOCHS_STAGE2} epochs...")
print(f"📊 Learning rate: 0.0001 (low for stable fine-tuning)")
print(f"🔓 ResNet50 base: PARTIALLY UNFROZEN (last 30 layers)")

start_time = time.time()

history_stage2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_STAGE2,
    callbacks=callbacks_stage2,
    verbose=1,
    class_weight=class_weight_dict
)

stage2_time = time.time() - start_time
print(f"\n✅ Stage 2 completed in {stage2_time:.2f} seconds ({stage2_time/60:.2f} minutes)")

# Get final results
stage2_val_auc = max(history_stage2.history['val_auc'])
total_time = stage1_time + stage2_time

print(f"\n📊 Training Summary:")
print(f"   Stage 1 best AUC: {stage1_val_auc:.4f}")
print(f"   Stage 2 best AUC: {stage2_val_auc:.4f}")
print(f"   Improvement: {stage2_val_auc - stage1_val_auc:+.4f}")
print(f"   Total time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

In [ ]:
# ============================================
# STEP 12: EVALUATE ON TEST SET
# ============================================
print("\n" + "="*60)
print("STEP 12: EVALUATING ON TEST SET")
print("="*60)

# Get predictions
print("🔮 Generating predictions on test set...")
y_test_proba = model.predict(test_generator, verbose=1)
y_test_pred = (y_test_proba > 0.5).astype(int).flatten()
y_test_true = test_generator.classes

# Calculate metrics
test_acc = accuracy_score(y_test_true, y_test_pred)
test_prec = precision_score(y_test_true, y_test_pred, zero_division=0)
test_rec = recall_score(y_test_true, y_test_pred, zero_division=0)
test_f1 = f1_score(y_test_true, y_test_pred, zero_division=0)
test_auc = roc_auc_score(y_test_true, y_test_proba)

print(f"\n📊 Test Set Results (threshold=0.5):")
print(f"   Accuracy:  {test_acc*100:.2f}%")
print(f"   Precision: {test_prec*100:.2f}%")
print(f"   Recall:    {test_rec*100:.2f}%")
print(f"   F1 Score:  {test_f1*100:.2f}%")
print(f"   ROC AUC:   {test_auc:.4f}")

# Comparison with VGG16
vgg16_auc = 0.706
improvement = test_auc - vgg16_auc
print(f"\n🆚 Comparison with VGG16:")
print(f"   VGG16 AUC:    {vgg16_auc:.4f}")
print(f"   ResNet50 AUC: {test_auc:.4f}")
print(f"   Improvement:  {improvement:+.4f} ({improvement/vgg16_auc*100:+.1f}%)")

if test_auc > vgg16_auc:
    print("🎉 ResNet50 outperformed VGG16!")
else:
    print("⚠️  ResNet50 underperformed VGG16")

In [ ]:
# ============================================
# STEP 13: THRESHOLD OPTIMIZATION
# ============================================
print("\n" + "="*60)
print("STEP 13: THRESHOLD OPTIMIZATION FOR MEDICAL USE")
print("="*60)

# Find optimal threshold for medical screening
precision, recall, thresholds = precision_recall_curve(y_test_true, y_test_proba)

# Medical target: 70% recall (catch 70% of DR cases)
target_recall = 0.70
recall_diffs = np.abs(recall - target_recall)
optimal_idx = np.argmin(recall_diffs)
optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5

print(f"🎯 Threshold optimization for medical screening:")
print(f"   Target recall: {target_recall*100:.0f}% (catch {target_recall*100:.0f}% of DR cases)")
print(f"   Optimal threshold: {optimal_threshold:.3f}")

# Test different thresholds
print(f"\n📊 Performance at different thresholds:")
print(f"{'Threshold':<10} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1':<10}")
print("-" * 50)

for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, optimal_threshold]:
    y_pred_thresh = (y_test_proba > thresh).astype(int).flatten()
    acc = accuracy_score(y_test_true, y_pred_thresh)
    prec = precision_score(y_test_true, y_pred_thresh, zero_division=0)
    rec = recall_score(y_test_true, y_pred_thresh, zero_division=0)
    f1 = f1_score(y_test_true, y_pred_thresh, zero_division=0)
    
    print(f"{thresh:<10.2f} {acc*100:<10.1f} {prec*100:<10.1f} {rec*100:<10.1f} {f1*100:<10.1f}")

# Recommended threshold results
y_pred_optimal = (y_test_proba > optimal_threshold).astype(int).flatten()
optimal_acc = accuracy_score(y_test_true, y_pred_optimal)
optimal_prec = precision_score(y_test_true, y_pred_optimal, zero_division=0)
optimal_rec = recall_score(y_test_true, y_pred_optimal, zero_division=0)
optimal_f1 = f1_score(y_test_true, y_pred_optimal, zero_division=0)

print(f"\n🏥 Recommended for medical use (threshold={optimal_threshold:.3f}):")
print(f"   Accuracy:  {optimal_acc*100:.2f}%")
print(f"   Precision: {optimal_prec*100:.2f}%")
print(f"   Recall:    {optimal_rec*100:.2f}% (detects {optimal_rec*100:.1f}% of DR cases)")
print(f"   F1 Score:  {optimal_f1*100:.2f}%")

if optimal_rec >= 0.6:
    print("✅ Clinically viable recall achieved!")
else:
    print("⚠️  Recall still below clinical threshold (60%+)")

In [ ]:
# ============================================
# STEP 14: VISUALIZATION & ANALYSIS
# ============================================
print("\n" + "="*60)
print("STEP 14: VISUALIZATION & ANALYSIS")
print("="*60)

# Combine training histories
combined_history = {
    'loss': history_stage1.history['loss'] + history_stage2.history['loss'],
    'val_loss': history_stage1.history['val_loss'] + history_stage2.history['val_loss'],
    'accuracy': history_stage1.history['accuracy'] + history_stage2.history['accuracy'],
    'val_accuracy': history_stage1.history['val_accuracy'] + history_stage2.history['val_accuracy'],
    'auc': history_stage1.history['auc'] + history_stage2.history['auc'],
    'val_auc': history_stage1.history['val_auc'] + history_stage2.history['val_auc']
}

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('ResNet50 Training Progress (Two-Stage Training)', fontsize=16)

# Loss plot
axes[0, 0].plot(combined_history['loss'], label='Train Loss', color='blue')
axes[0, 0].plot(combined_history['val_loss'], label='Val Loss', color='orange')
axes[0, 0].axvline(x=len(history_stage1.history['loss'])-1, color='red', linestyle='--', alpha=0.7, label='Stage 1→2')
axes[0, 0].set_title('Model Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy plot
axes[0, 1].plot(combined_history['accuracy'], label='Train Accuracy', color='blue')
axes[0, 1].plot(combined_history['val_accuracy'], label='Val Accuracy', color='orange')
axes[0, 1].axvline(x=len(history_stage1.history['accuracy'])-1, color='red', linestyle='--', alpha=0.7, label='Stage 1→2')
axes[0, 1].set_title('Model Accuracy')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# AUC plot
axes[1, 0].plot(combined_history['auc'], label='Train AUC', color='blue')
axes[1, 0].plot(combined_history['val_auc'], label='Val AUC', color='orange')
axes[1, 0].axvline(x=len(history_stage1.history['auc'])-1, color='red', linestyle='--', alpha=0.7, label='Stage 1→2')
axes[1, 0].set_title('Model AUC')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('AUC')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Confusion Matrix
cm = confusion_matrix(y_test_true, y_pred_optimal)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1])
axes[1, 1].set_title(f'Confusion Matrix (threshold={optimal_threshold:.3f})')
axes[1, 1].set_xlabel('Predicted Label')
axes[1, 1].set_ylabel('True Label')
axes[1, 1].set_xticklabels(['No DR', 'Has DR'])
axes[1, 1].set_yticklabels(['No DR', 'Has DR'])

plt.tight_layout()
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test_true, y_test_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='orange', lw=2, label=f'ROC (AUC = {test_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - ResNet50')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

print("📊 Visualizations complete!")

In [ ]:
# ============================================
# STEP 15: FINAL RESULTS SUMMARY
# ============================================
print("\n" + "="*60)
print("STEP 15: FINAL RESULTS SUMMARY")
print("="*60)

print(f"🏆 ResNet50 Two-Stage Training Results:")
print(f"\n📊 Model Performance:")
print(f"   Architecture: ResNet50 + Custom Head (512→256→128→1)")
print(f"   Total Parameters: {total_params:,}")
print(f"   Training Strategy: Two-stage (frozen → fine-tuned)")
print(f"   Loss Function: Focal Loss (γ=2.0, α=0.75)")

print(f"\n📈 Training Results:")
print(f"   Stage 1 (frozen): {len(history_stage1.history['loss'])} epochs, AUC {stage1_val_auc:.4f}")
print(f"   Stage 2 (fine-tuned): {len(history_stage2.history['loss'])} epochs, AUC {stage2_val_auc:.4f}")
print(f"   Total training time: {total_time/60:.1f} minutes")

print(f"\n🎯 Test Set Performance:")
print(f"   Default threshold (0.5):")
print(f"     AUC: {test_auc:.4f}")
print(f"     Accuracy: {test_acc*100:.1f}%")
print(f"     Precision: {test_prec*100:.1f}%")
print(f"     Recall: {test_rec*100:.1f}%")
print(f"     F1: {test_f1*100:.1f}%")

print(f"\n🏥 Medical threshold ({optimal_threshold:.3f}):")
print(f"     Accuracy: {optimal_acc*100:.1f}%")
print(f"     Precision: {optimal_prec*100:.1f}%")
print(f"     Recall: {optimal_rec*100:.1f}% (detects {optimal_rec*100:.0f}% of DR cases)")
print(f"     F1: {optimal_f1*100:.1f}%")

print(f"\n🆚 Comparison with Previous Models:")
print(f"   Random Forest: AUC 0.551, Recall 60.2%")
print(f"   SVM:          AUC 0.505, Recall 58.8%")
print(f"   VGG16:        AUC 0.706, Recall 1.0%")
print(f"   ResNet50:     AUC {test_auc:.3f}, Recall {optimal_rec*100:.1f}%")

# Determine best model
models_comparison = [
    ('Random Forest', 0.551, 60.2),
    ('SVM', 0.505, 58.8),
    ('VGG16', 0.706, 1.0),
    ('ResNet50', test_auc, optimal_rec*100)
]

best_auc_model = max(models_comparison, key=lambda x: x[1])
best_recall_model = max(models_comparison, key=lambda x: x[2])

print(f"\n🏆 Best Models:")
print(f"   Highest AUC: {best_auc_model[0]} ({best_auc_model[1]:.3f})")
print(f"   Highest Recall: {best_recall_model[0]} ({best_recall_model[2]:.1f}%)")

if test_auc >= 0.75 and optimal_rec >= 0.65:
    print(f"\n✅ CLINICAL VIABILITY: ResNet50 achieves both good AUC (≥0.75) and recall (≥65%)")
    print(f"   Recommended for medical screening with threshold {optimal_threshold:.3f}")
elif test_auc >= 0.70:
    print(f"\n⚠️  PARTIAL SUCCESS: Good AUC but may need recall improvement")
else:
    print(f"\n❌ NEEDS IMPROVEMENT: Consider XGBoost or CNN Baseline next")

print(f"\n💾 Model saved as: best_resnet50_stage2.h5")
print(f"📁 Use this model for inference with threshold {optimal_threshold:.3f}")

## 🎯 Experiment Summary

### Key Achievements:
1. **Two-stage training**: Frozen base → Fine-tuned (stable learning)
2. **Focal loss**: Better handling of class imbalance than VGG16
3. **Medical threshold optimization**: Prioritizing recall for clinical use
4. **ResNet50 architecture**: Modern skip connections for medical features

### Next Steps:
- If AUC ≥ 0.75: **SUCCESS** - Use this model for deployment
- If AUC < 0.75: Try **XGBoost** or **CNN Baseline** experiments
- Consider ensemble methods combining top 2-3 models

### For Production Use:
```python
# Load model
model = tf.keras.models.load_model('best_resnet50_stage2.h5', 
                                   custom_objects={'focal_loss_fixed': focal_loss(2.0, 0.75)})

# Predict with optimized threshold
predictions = model.predict(images)
binary_predictions = (predictions > optimal_threshold).astype(int)
```